In [1]:
import requests
import pandas 
import numpy 
import json


In [2]:
import requests
import pandas 
import numpy 
import json
from io import StringIO

def json_to_series(text):
    keys, values = zip(*[(dct['label'], dct['value']) for dct in json.loads(text)])
    return pd.Series(values, index=keys)


index = 1
req_total = []
dados_veiculos = pandas.DataFrame()

while index < 150:
    req_veiculos = requests.get("https://www.webmotors.com.br/api/search/car?url=https://www.webmotors.com.br/carros%2Fms%3Fanunciante%3DConcession%25C3%25A1ria%257CLoja%26estadocidade%3DMato%2520Grosso%2520do%2520Sul", params="actualPage=" + str(index))
    req_veiculos = json.loads(req_veiculos.text)
    try:
        req_veiculos = req_veiculos['SearchResults']
        json_str = json.dumps(req_veiculos)
        veiculos = pandas.read_json(json_str)
        tabela_especificacoes = pandas.json_normalize(veiculos['Specification'])
        veiculos = veiculos.join(other= tabela_especificacoes)

        tabela_precos = pandas.json_normalize(veiculos['Prices'])
        veiculos = veiculos.join(other= tabela_precos)
        
        tabela_vendedor = pandas.json_normalize(veiculos['Seller'])
        veiculos = veiculos.join(other= tabela_vendedor)
        dados_veiculos = pandas.concat([dados_veiculos, veiculos])
    except:
        print(f"Ocorreu um erro na página {index}")
    index +=1

dados_veiculos = dados_veiculos.drop(labels=['Prices', 'Channels', 'VehicleAttributes', 'Media', 'Specification', 'Seller'], axis=1)

dados_veiculos.to_csv('output/anuncios_webmotors.csv')

Ocorreu um erro na página 54
Ocorreu um erro na página 74
Ocorreu um erro na página 90
Ocorreu um erro na página 91
Ocorreu um erro na página 92
Ocorreu um erro na página 93
Ocorreu um erro na página 94
Ocorreu um erro na página 95
Ocorreu um erro na página 96
Ocorreu um erro na página 97
Ocorreu um erro na página 98
Ocorreu um erro na página 99
Ocorreu um erro na página 100
Ocorreu um erro na página 101
Ocorreu um erro na página 102
Ocorreu um erro na página 103
Ocorreu um erro na página 104
Ocorreu um erro na página 105
Ocorreu um erro na página 106
Ocorreu um erro na página 107
Ocorreu um erro na página 108
Ocorreu um erro na página 109
Ocorreu um erro na página 110
Ocorreu um erro na página 111
Ocorreu um erro na página 112
Ocorreu um erro na página 113
Ocorreu um erro na página 114
Ocorreu um erro na página 115
Ocorreu um erro na página 116
Ocorreu um erro na página 117
Ocorreu um erro na página 118
Ocorreu um erro na página 119
Ocorreu um erro na página 120
Ocorreu um erro na pág